In [1]:
pip install transformers accelerate pillow torch torchvision bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 52.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 51.0 MB/s  0:00:01 eta 0:00:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.14.0
    Uninstalling sympy-1.14.0:
      Successfully uninstalled sympy-1.14.0━━━━━ 0/2 [sympy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [bitsandbytes] [bitsandbytes]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from torchvision import transforms, models
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq


device = "cuda" if torch.cuda.is_available() else "cpu"

num_classes = 13
class_names = [
    "ABAP Dictionary",
    "BW4Cockpit (Stammdaten)",
    "Bewegungsdaten",
    "Composite Provider",
    "DTP",
    "Data Flow Object",
    "Data Mart",
    "Data Source",
    "Data Store Object",
    "Datenvorschau",
    "Excel",
    "Query",
    "Transformationen"
]

resnet = models.resnet18(weights=None)
resnet.fc = torch.nn.Linear(resnet.fc.in_features, num_classes)
resnet.load_state_dict(torch.load("outputs/resnet_best.pt", map_location=device))
resnet = resnet.to(device)
resnet.eval()


model_id = "Qwen/Qwen2.5-VL-7B-Instruct"

processor = AutoProcessor.from_pretrained(model_id, token="none")

qwen = AutoModelForVision2Seq.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto",
    token="none"
)


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

def classify_image(image_path):
    img = Image.open(image_path).convert("RGB")
    tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = resnet(tensor)
        _, pred = torch.max(outputs, 1)

    return class_names[pred.item()], img


PROMPTS = {
    "Excel": """
Extrahiere die tabellarischen Daten aus dem Excel-Screenshot.
Ignoriere UI-Elemente wie Menüleisten, Filterbereiche, Metadaten oder Seitentitel.
Konzentriere dich ausschließlich auf die sichtbare Tabelle.

Gib NUR gültiges JSON zurück.
KEIN Markdown.
KEINE ```json Blöcke.
KEINE Erklärungen.
KEINE Kommentare.
KEINE zusätzlichen Texte.

Format:

{
  "columns": ["Spalte1", "Spalte2", ...],
  "rows": [
    ["Wert1", "Wert2", ...],
    ["Wert1", "Wert2", ...]
  ]
}
""",

    "DTP": """
Analysiere den Screenshot eines Data Transfer Process (DTP) in SAP BW/4HANA.

Extrahiere ausschließlich die für einen DTP relevanten Informationen:

- DTP-Name bzw. technische ID
- Ausführungsmodus (z. B. Serial SAP HANA Execution, Dialog, Hintergrund)
- Quelle des DTP (z. B. DataSource, RSDS, Datei)
- Zielobjekt (z. B. ADSO, InfoCube)
- Sichtbare Prozessschritte (z. B. Fill data transfer intermediate storage, Prepare for Extraction, Data Package Loop)
- Hinweise auf die Extraktionsart (z. B. Datei-Extraktion)
- Sichtbare Systemmeldungen oder Pop-ups (z. B. SAP GUI Security File Access)

Gib NUR gültiges JSON zurück.
KEIN Markdown.
KEINE ```json Blöcke.
KEINE Erklärungen.
KEINE Kommentare.
KEINE zusätzlichen Texte.

Format:

{
  "dtp_name": "",
  "execution_mode": "",
  "source": "",
  "target": "",
  "process_steps": [],
  "extraction_type": "",
  "system_messages": []
}
""",

        "Transformationen": """
Analysiere den Screenshot einer Transformation in SAP BW/4HANA.

Extrahiere ausschließlich die variablen, inhaltlichen Informationen der Transformation:

- Name bzw. technische ID der Quelle
- Name bzw. technische ID des Ziels
- Alle Quellfelder (Name + Datentyp)
- Alle Zielfelder (Name + Datentyp)
- Alle sichtbaren Feldzuordnungen (source_field → target_field)
- Nur echte Mappings, keine Linieninterpretation
- Behandle auch farbige, dünne oder schwach sichtbare Linien als gültige Mappings, wenn sie eine Verbindung zwischen zwei Feldern darstellen.
- Keine automatisch generierten Felder erfinden

Gib NUR gültiges JSON zurück.
KEIN Markdown.
KEINE ```json Blöcke.
KEINE Erklärungen.
KEINE Kommentare.
KEINE zusätzlichen Texte.

Format:

{
  "source": {
    "name": "",
    "fields": [
      {"name": "", "type": ""}
    ]
  },
  "target": {
    "name": "",
    "fields": [
      {"name": "", "type": ""}
    ]
  },
  "mappings": [
    {"source_field": "", "target_field": ""}
  ]
}
"""
}


def qwen_extract(image, prompt):
    print("[QWEN] Baue messages...")
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": prompt}
            ]
        }
    ]

    print("[QWEN] Wende chat template an...")
    text_prompt = processor.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    print("[QWEN] Erzeuge Inputs...")
    inputs = processor(
        text=text_prompt,
        images=image,
        return_tensors="pt"
    ).to(device)

    for k, v in inputs.items():
        print(f"[QWEN] {k} shape:", tuple(v.shape))

    print("[QWEN] Starte generate()...")
    output = qwen.generate(
        **inputs,
        max_new_tokens=512,   # jetzt aktiv
        do_sample=False,
        temperature=0.0,
        eos_token_id=None      # jetzt aktiv
    )
    print("[QWEN] generate() fertig.")

    decoded = processor.batch_decode(output, skip_special_tokens=True)[0]
    print("[QWEN] Decoding fertig.")
    return decoded


def process_image(image_path):
    pred_class, img = classify_image(image_path)
    print(f"ResNet-Klasse: {pred_class}")

    prompt = PROMPTS[pred_class]
    json_output = qwen_extract(img, prompt)

    print("\nJSON-Ausgabe:")
    print(json_output)
    return json_output


if __name__ == "__main__":
    process_image("/workspace/Projekt/Testdaten/Data/Ungelabelt/03b359fa5d3d48148433b6d78a2e3cfa.png")

2026-01-12 19:14:17.253429: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/tmp/ipykernel_1685/1014933687.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


ResNet-Klasse: Excel
[QWEN] Baue messages...
[QWEN] Wende chat template an...
[QWEN] Erzeuge Inputs...
[QWEN] input_ids shape: (1, 1506)
[QWEN] attention_mask shape: (1, 1506)
[QWEN] pixel_values shape: (5336, 1176)
[QWEN] image_grid_thw shape: (1, 3)
[QWEN] Starte generate()...
[QWEN] generate() fertig.
[QWEN] Decoding fertig.

JSON-Ausgabe:
system
You are a helpful assistant.
user

Extrahiere die tabellarischen Daten aus dem Excel-Screenshot.
Ignoriere UI-Elemente wie Menüleisten, Filterbereiche, Metadaten oder Seitentitel.
Konzentriere dich ausschließlich auf die sichtbare Tabelle.

Gib NUR gültiges JSON zurück.
KEIN Markdown.
KEINE ```json Blöcke.
KEINE Erklärungen.
KEINE Kommentare.
KEINE zusätzlichen Texte.

Format:

{
  "columns": ["Spalte1", "Spalte2", ...],
  "rows": [
    ["Wert1", "Wert2", ...],
    ["Wert1", "Wert2", ...]
  ]
}

assistant
{
  "columns": ["Product", "Deviation %", "Revenue $", "Revenue 1. Quartal 2018 EUR", "Revenue 1. Quartal 2019 EUR", "Revenue 2018 EUR", 